In [8]:
from pyspark.sql.functions import lit
from pyspark.sql import SparkSession, functions as F
import pandas as pd
from packages.utils.transformations import MyTransformation as mtf
from packages.utils.spark_udfs import soundex_udf
from pyspark.sql.functions import col, array_contains


spark = SparkSession.builder.master("local[*]").appName("example2").getOrCreate()

# Read and rename columns as before
def read_with_origin(path, origin):
    df = spark.read.csv(path, header=None, inferSchema=True).select(["_c0","_c1","_c2","_c3","_c4","_c5"])
    for i in range(6):
        df = df.withColumnRenamed(f"_c{i}", str(i))
    df = df.withColumn("origin", lit(origin))
    return df

In [9]:
_expected = {'gt': 25000, 'tp': 24550, 'fp': 12333, 'fn': 450}
match_column = "0"

df1 = read_with_origin("data/df1.csv", "df1")
df2 = read_with_origin("data/df2.csv", "df2")
df3 = read_with_origin("data/df3.csv", "df3")
df4 = read_with_origin("data/df4.csv", "df4")
df5 = read_with_origin("data/df5.csv", "df5")

for col_name in df1.columns:
    if col_name != match_column:
        df1 = df1.withColumn(str(col_name), soundex_udf(df1[col_name]))
        df2 = df2.withColumn(str(col_name), soundex_udf(df2[col_name]))
        df3 = df3.withColumn(str(col_name), soundex_udf(df3[col_name]))
        df4 = df4.withColumn(str(col_name), soundex_udf(df4[col_name]))
        df5 = df5.withColumn(str(col_name), soundex_udf(df5[col_name]))
        
# Concatenate and drop duplicates
spark_df1 = df1
spark_df2 = df2.union(df3).union(df4).union(df5)

In [15]:
cols = ['1', '2', '3', '4', '5']

ground_truth_ids = spark_df1.join(spark_df2, on=["0"], how="inner").select(F.col("0").alias("df1_id")).distinct()
ground_truth_ids.cache()
gt = ground_truth_ids.count()

# 1. Create entity keys for df1
df1_entities = spark_df1.withColumn(
    "entity_key", F.concat_ws("", *[F.col(c) for c in cols])
).select(F.col("0").alias("df1_id"), "entity_key", *cols)

# 2. Create entity keys for the bucket (use spark_df2 or concatData)
bucket_entities = spark_df2.withColumn(
    "entity_key", F.concat_ws("", *[F.col(c) for c in cols])
).select(
    F.col("0").alias("bucket_id"),
    "entity_key",
    *cols
)

# 3. Cross join and count matches
joined = df1_entities.alias("a").crossJoin(bucket_entities.alias("b"))
match_exprs = [(F.col(f"a.{col}") == F.col(f"b.{col}")).cast("int") for col in cols]
joined = joined.withColumn("match_count", sum(match_exprs))

# 4. Filter pairs with at least 3 matching columns
filtered = joined.filter(F.col("match_count") >= 3)

# 5. For each df1_id, find the max match_count
max_match = filtered.groupBy("a.df1_id").agg(F.max("match_count").alias("max_match_count")).withColumnRenamed("df1_id", "max_df1_id")

# 6. Join back to get all (df1_id, bucket_id) pairs with the max match_count
best_matches = filtered.join(
    max_match,
    (filtered["a.df1_id"] == max_match["max_df1_id"]) & (filtered["match_count"] == max_match["max_match_count"])
).select(
    F.col("a.df1_id").alias("df1_id"),
    F.col("b.bucket_id").alias("bucket_id"),
    F.col("match_count")
)

# 7. Group by bucket_id to collect all df1_ids assigned to it
buckets = best_matches.groupBy("bucket_id").agg(
    F.collect_list("df1_id").alias("assigned_df1_ids")
)

buckets.cache()

DataFrame[bucket_id: string, assigned_df1_ids: array<string>]

In [16]:
# True Positives
tp_arr = buckets \
    .filter(array_contains(col("assigned_df1_ids"), col("bucket_id"))) \
    .join(ground_truth_ids, buckets.bucket_id == ground_truth_ids.df1_id, how="inner")

tp = tp_arr.count()

fp = buckets.filter(~array_contains(col("assigned_df1_ids"), col("bucket_id"))).count()
fn = gt - tp

precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0

print(_expected)
print(f"Ground Truth (gt): {gt}")
print(f"True Positives (tp): {tp}")
print(f"False Positives (fp): {fp}")
print(f"False Negative (fn): {fn}")
print(f"precision: {precision}")
print(f"Recall: {recall}")

{'gt': 5, 'tp': 4, 'fp': 2, 'fn': 1}
Ground Truth (gt): 5
True Positives (tp): 4
False Positives (fp): 2
False Negative (fn): 1
precision: 0.6666666666666666
Recall: 0.8


In [20]:
buckets.show()

+---------+--------------------+
|bucket_id|    assigned_df1_ids|
+---------+--------------------+
| AA100187|          [AA100187]|
| AA100252|          [AA100252]|
| AA100290|          [AA100290]|
| AA100327|          [AA100327]|
| AA100360| [AA81619, AA100360]|
| AA100381|          [AA100381]|
| AA100422|          [AA100422]|
|  AA10048|[AA97299, AA11609...|
| AA100508|          [AA100508]|
| AA100514|          [AA100514]|
| AA100613|[AA192247, AA100613]|
| AA100621|          [AA100621]|
| AA100654|          [AA100654]|
| AA100727|[AA100727, AA196019]|
| AA100824|          [AA100824]|
| AA100835|          [AA100835]|
| AA100888|          [AA100888]|
| AA100904|          [AA100904]|
| AA100932|          [AA100932]|
| AA100966|          [AA100966]|
+---------+--------------------+
only showing top 20 rows



In [13]:
buckets.write.mode("overwrite").parquet("data/output/buckets1.parquet")